## Work
1. 請嘗試將 preproc_x 替換成以每筆資料的 min/max 進行標準化至 -1 ~ 1 間，再進行訓練
2. 請嘗試將 mlp 疊更深 (e.g 5~10 層)，進行訓練後觀察 learning curve 的走勢
3. (optional) 請改用 GPU 進行訓練 (如果你有 GPU 的話)，比較使用 CPU 與 GPU 的訓練速度

In [1]:
import os
import keras

# Try to enable GPU (optional)
os.environ["CUDA_VISIBLE_DEVICES"] = ""

Using TensorFlow backend.


In [2]:
train, test = keras.datasets.cifar10.load_data()

In [3]:
## 資料前處理
"""
Your code here
"""
def preproc_x(x, flatten = True):
    x = x.reshape((len(x), -1))
    x = (x - x.min(axis=0)) / (x.max(axis=0) - x.min(axis=0)) * 2 - 1
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y

In [4]:
x_train, y_train = train
x_test, y_test = test

# Preproc the inputs
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# Preprc the outputs
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [7]:
"""
Your code here
"""
def build_mlp(input_shape, output_units=10, num_neurons=[512, 256, 128, 64, 32]):
    input_layer = keras.layers.Input(input_shape)
    for i, n_units in enumerate(num_neurons):
        if i == 0:
            x = keras.layers.Dense(units = n_units, activation='relu', name="hidden_layer"+str(i+1))(input_layer)
        else:
            x = keras.layers.Dense(units = n_units, activation='relu', name="hidden_layer"+str(i+1))(x)
    
    out = keras.layers.Dense(units = output_units, activation='relu', name="output")(x)
    model = keras.models.Model(inputs=[input_layer], outputs=[out])        
    return model
model = build_mlp(input_shape = x_train.shape[1:])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 3072)              0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               131328    
_________________________________________________________________
hidden_layer3 (Dense)        (None, 128)               32896     
_________________________________________________________________
hidden_layer4 (Dense)        (None, 64)                8256      
_________________________________________________________________
hidden_layer5 (Dense)        (None, 32)                2080      
_________________________________________________________________
output (Dense)               (None, 10)                330       
Total para

In [8]:
## 超參數設定
LEARNING_RATE = 0.001
EPOCHS = 100
BATCH_SIZE = 256

In [9]:
optimizer = keras.optimizers.Adam(lr=LEARNING_RATE)
model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optimizer)

In [10]:
model.fit(x_train, y_train, 
          epochs=EPOCHS, 
          batch_size=BATCH_SIZE, 
          validation_data=(x_test, y_test), 
          shuffle=True)

Train on 50000 samples, validate on 10000 samples
Epoch 1/100
50000/50000 [==============================] - 22s 442us/step - loss: 2.6775 - acc: 0.2228 - val_loss: 2.0786 - val_acc: 0.3064
Epoch 2/100
50000/50000 [==============================] - 13s 267us/step - loss: 1.9966 - acc: 0.3165 - val_loss: 1.9773 - val_acc: 0.3026
Epoch 3/100
50000/50000 [==============================] - 10s 208us/step - loss: 2.0130 - acc: 0.2793 - val_loss: 2.2456 - val_acc: 0.1565
Epoch 4/100
50000/50000 [==============================] - 11s 212us/step - loss: 2.0669 - acc: 0.2409 - val_loss: 1.9589 - val_acc: 0.3142
Epoch 5/100
50000/50000 [==============================] - 10s 209us/step - loss: 1.9919 - acc: 0.2967 - val_loss: 2.0492 - val_acc: 0.2349
Epoch 6/100
50000/50000 [==============================] - 10s 208us/step - loss: 1.9156 - acc: 0.3169 - val_loss: 1.8877 - val_acc: 0.3122
Epoch 7/100
50000/50000 [==============================] - 11s 214us/step - loss: 2.0934 - acc: 0.2235 - val_l

Epoch 59/100
50000/50000 [==============================] - 12s 234us/step - loss: 1.5947 - acc: 0.4087 - val_loss: 1.6525 - val_acc: 0.4021
Epoch 60/100
50000/50000 [==============================] - 10s 209us/step - loss: 1.6211 - acc: 0.4060 - val_loss: 1.6480 - val_acc: 0.4102
Epoch 61/100
50000/50000 [==============================] - 10s 191us/step - loss: 1.5611 - acc: 0.4299 - val_loss: 1.6258 - val_acc: 0.4226
Epoch 62/100
50000/50000 [==============================] - 9s 188us/step - loss: 1.7187 - acc: 0.3724 - val_loss: 1.7209 - val_acc: 0.3919
Epoch 63/100
50000/50000 [==============================] - 10s 210us/step - loss: 1.6336 - acc: 0.4085 - val_loss: 1.7345 - val_acc: 0.3966
Epoch 64/100
50000/50000 [==============================] - 10s 204us/step - loss: 1.6007 - acc: 0.4225 - val_loss: 1.6446 - val_acc: 0.4216
Epoch 65/100
50000/50000 [==============================] - 10s 197us/step - loss: 1.5368 - acc: 0.4383 - val_loss: 1.6263 - val_acc: 0.4374
Epoch 66/100
5

In [12]:
import matplotlib.pyplot as plt
%matplotlib inline


train_loss = model.history.history["loss"]
valid_loss = model.history.history["val_loss"]

train_acc = model.history.history["acc"]
valid_acc = model.history.history["val_acc"]

plt.plot(range(len(train_loss)), train_loss, label="train loss")
plt.plot(range(len(valid_loss)), valid_loss, label="valid loss")
plt.legend()
plt.title("Loss")
plt.show()

plt.plot(range(len(train_acc)), train_acc, label="train accuracy")
plt.plot(range(len(valid_acc)), valid_acc, label="valid accuracy")
plt.legend()
plt.title("Accuracy")
plt.show()